# Lab 7: Multiple tests
We generate gene expression data for 4 genes

In [1]:
data = data.frame(group=c(rep(0,10),rep(1,10)), 
                  gene1 = c(rnorm(10, mean=0.2, sd=1), rnorm(10, mean=2.2, sd=1)), 
                  gene2 = c(rnorm(10, mean=0.2, sd=1), rnorm(10, mean=0.2, sd=1)), 
                  gene3 = c(rnorm(10, mean=0.2, sd=1), rnorm(10, mean=0.2, sd=1)), 
                  gene4 = c(rnorm(10, mean=0.2, sd=1), rnorm(10, mean=2.2, sd=1)))
data

group,gene1,gene2,gene3,gene4
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0.5897151,0.99987821,-0.19222842,-0.4033234
0,0.3879895,0.39597085,0.30894302,3.3157428
0,1.0706384,-0.07572495,0.46094664,1.1563011
0,0.6050366,-0.30761677,-0.12734170,-1.0653760
0,-1.2725085,0.05514196,-0.38070297,0.4696083
0,-0.3114632,-0.74945140,1.67113364,-0.2614761
0,1.5862377,0.02334721,-0.78842792,-1.2046106
0,0.0878046,1.27897779,1.18659737,0.9487144
0,0.8659912,0.48403517,0.60616226,0.2109329


## Bonferroni correction

In [2]:
numtest = 4
pvalue = 1:numtest
for(i in 1:numtest){
    pvalue[i] = t.test(data[,i+1] ~ data[,1])$p.value
}

print("the Bonferroni adjusted pvalues")
pvalue*numtest

[1] "the Bonferroni adjusted pvalues"


[1] 0.000621039 3.509097315 3.844983766 0.012528536

## False Discovery Rate (FDR)
To perform the FDR analysis in R, we need to install the package **fuzzySim** by typing **install.packages("fuzzySim")** in R.

In [3]:
library(fuzzySim)
data = data.frame(group=c(rep(0,10),rep(1,10)), 
                  gene1 = c(rnorm(10, mean=0.2, sd=1), rnorm(10, mean=2.2, sd=1)), 
                  gene2 = c(rnorm(10, mean=0.2, sd=1), rnorm(10, mean=0.2, sd=1)), 
                  gene3 = c(rnorm(10, mean=0.2, sd=1), rnorm(10, mean=0.2, sd=1)), 
                  gene4 = c(rnorm(10, mean=0.2, sd=1), rnorm(10, mean=2.2, sd=1)))
data

FDR(data = data, sp.cols = 1, var.cols = 2:5, family = "gaussian")

ERROR: Error in library(fuzzySim): there is no package called ‘fuzzySim’


## Microarray gene expression analysis

To analyze gene expression data, we need to install Bioconductor packages. To install Bioconductor packages, type the following in an R command window:

```{code}
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(c("affy","limma","arrays"))

```

1. We first import data from the package **arrays**.

In [ ]:
library(affy)   # Affymetrix pre-processing
library(limma)  # two-color pre-processing; differential
                  # expression
                
## import "phenotype" data, describing the experimental design
phenoData <- read.AnnotatedDataFrame(system.file("extdata", "pData.txt", package="arrays"))
celfiles <- system.file("extdata", package="arrays")

2. RMA normalization for microarray expression data

In [ ]:
eset <- justRMA(phenoData=phenoData, celfile.path=celfiles)

3. Identify differentially expressed genes

In [ ]:
combn <- factor(paste(pData(phenoData)[,1], pData(phenoData)[,2], sep = "_"))
design <- model.matrix(~combn) # describe model to be fit

fit <- lmFit(eset, design)  # fit each probeset to model
efit <- eBayes(fit)        # empirical Bayes adjustment
topTable(efit, coef=2)      # table of differentially expressed probesets